<a href="https://colab.research.google.com/github/FoleyLab/quantum_twitch/blob/master/split_operator_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Authors** [Jason Codrington](https://njsecretaryhighereducation.com/2017/04/20/apr2017-wpu/) and [Jay Foley](https://foleylab.github.io/)

### **[Background](https://arxiv.org/pdf/1306.3247.pdf)**
We are going to demonstrate the use of the split-operator method to solve the time-dependent Schroedinger equation in 1D,

$$ i\hbar \frac{\partial}{\partial t} \Psi(x,t) = \hat{H} \Psi(x,t), $$

where 

$$ \hat{H} = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + V(x) \equiv \hat{K} + \hat{V} $$.

The idea behind the split operator method is to approximate the time evolution operator $\hat{U}(t,t_0)$ for a very small time-increment $\Delta t$ as follows:

$$ \hat{U}(t+\Delta t, t) = {\rm exp}\left( -\frac{i \Delta t}{2\hbar} \hat{V}\right) 
{\rm exp}\left( -\frac{i\Delta t}{\hbar} \hat{K} \right) 
{\rm exp}\left( - \frac{i \Delta t}{2\hbar} \hat{V} \right) $$,
such that 

$$ \Psi(x, t+\Delta t) \approx \hat{U}(t+\Delta t, t) \Psi(x,t). $$
In this approach, we apply a half time-step update to the wavefunction through the kinetic energy operator, a full time-step update through the potential energy operator, then a second half time-step update through the kinetic energy operator.  

That looks great, but how the heck do we exponentiate $\hat{K}$ with it's second derivatives?  IDK, but actually we don't have to.  The kinetic energy operator Fourier transforms to a quadratic function in momentum space:

$$ -\frac{\hbar^2}{2m}\frac{d^2}{dx^2}  \xrightarrow{\mathcal{F}}  \frac{ p^2}{2m}, $$

and this can be easily exponentiated.  We will denote the kinetic energy operator in momentum space as $ \hat{K}_p = \frac{p^2}{2m} $, and we can therefore write the following:

$$ {\rm exp}\left( -\frac{i \Delta t}{\hbar} \hat{K}\right) \Psi(x,t) = \mathcal{F}^{-1}\left( {\rm exp}\left( -\frac{i \Delta t}{\hbar} \hat{K}_p \right) \mathcal{F} \left( \Psi(x,t) \right)  \right). $$

For our purposes, this is actually awesome because we can take the Fourier and inverse Fourier transform of things with like a single line of python code.  The only mild additional discomfort we have to endure is having to carry an additional grid around - we will need a spatial grid to evaluate $\Psi(x,t)$ and $\hat{V}$ on, but we will need a momentum grid to evaluate $\hat{K}_p$ on.

We can define the points on our spatial grid as follows:

$$ x_j = x_{min} + j \Delta x $$ for $ j = 0, 1, ..., N-1 $ where $N$ is the number of gridpoints and 

$$ \Delta x = \frac{x_{max} - x_{min}}{N-1}$$.  

Our momentum grid will be defined as follows:

$$ p_j = j \frac{2 \pi \hbar}{N \Delta x}, $$
where $j = -\frac{N}{2}, ..., \frac{N}{2}. $

### **Outline of our code**

1. Add attributes to our class for $x_{min}$, $x_{max}$, $N$ and create grids

2. Add attributes to our class for other parts of the simulation, including 
   - time step $\Delta t$ (in atomic units)
   - mass of particle $m$ (in atomic units)
   - value of $\hbar$ (in atomic units)

3. Define initial wavefunction 
$$\Psi(x,t_0) = \frac{1}{\sigma \sqrt{2\pi}} {\rm exp}\left(\frac{-1}{2}\left(\frac{x-x_0}{\sigma}\right)^2\right) {\rm exp}\left(i k_0 x \right)$$
   - $k_0 \approx \frac{m}{ 2}$

   - $x_0 \approx \frac{x_{max}}{2}$

   - $\sigma \approx \frac{x_{max}-x_{min}}{30} $ 

4. Define $\hat{V}$... could be
   - Nothing
   - Square barrier
   - Square well
   - Gaussian barrier?
   - Harmonic well?

5. Define $\hat{K}_p$ operator using $\frac{{\bf p}^2}{2m}$

6. Form exponentials of the $\hat{V}$ and $\hat{K}_p$ operators
   - ${\bf K} = {\rm exp}\left( -\frac{i \Delta t}{\hbar} \hat{K}_p \right) $
   
   - ${\bf V} = {\rm exp}\left( -\frac{i \Delta t}{2\hbar} \hat{V} \right) $

7. Implement split operator method:
  - $\Psi^{(1)}(x,t) = {\bf V} \: \Psi(x,t) $

  - $\Psi^{(1)}(p,t) = \mathcal{F} \left( \Psi^{(1)}(x,t)\right)$

  - $\Psi^{(2)}(p,t) = {\bf K} \: \Psi^{(1)}(p,t)$

  - $\Psi^{(2)}(x,t) = \mathcal{F}^{-1} \left( \Psi^{(2)}(p,t)\right) $

  - $\Psi(x,t+\Delta t) = {\bf V} \: \Psi^{(2)}(x,t) $




In [ ]:

from matplotlib import animation, rc, pyplot as plt
from IPython.display import HTML
import numpy as np

class Quantum:
    def __init__(self, args):

        if 'box_length' in args:
            self.L = args['box_length']
        else:
            self.L = 10.0
        if 'time_step' in args:
            self.dt = args['time_step']
        else:
            self.dt = 0.1          
        if 'grid_points' in args:
            self.grid_points =args['grid_points']
        else:
            self.grid_points = 256
        if 'mass' in args:
          self.m = args['mass']
        else:
          self.m = 1

        ''' some constants and parameters in atomic units '''
        # reduced planck's constant
        self.hbar = 1
        # reduced mass for PIR system or a very light harmonic oscillator!
        self.mu = 1
        # very small spring constant!
        self.k = 1
        # position-space spread for a gaussian wavepacket
        self.sigma = self.L / 5
        # central position value for a gaussian wavepacket
        self.x0 = self.L / 2
        # central momentum for a gaussian wavepacket
        self.k0 = 0.4
        self.x = np.linspace(0, self.L, self.grid_points)
        self.dx = self.x[1]-self.x[0]
        res = self.grid_points 
        self.dk = 2 * np.pi / (res * self.dx)
        self.k = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dk
            
        self.V = np.zeros_like(self.x)
        #self.V = 0.5 * (self.x - self.voffset) ** 2
        self.Psi = np.sqrt(2/self.L) * np.sin((np.pi * self.x/self.L), dtype=complex)
        self.K = np.exp(-1/(2*self.m) * (self.k ** 2) * self.dt * 1j)
        self.R = np.exp(-0.5 * self.V * self.dt * 1j)
            
            
        



            
    def build_operator(self):
        self.R = np.exp(-0.5 * self.V * self.dt * 1j)
    
    def split_op(self):
        

        # Half-step in real space
        self.Psi *= self.R

        # FFT to momentum space
        self.Psi = np.fft.fft(self.Psi)

        # Full step in momentum space
        self.Psi *= self.K

        # iFFT back
        self.Psi = np.fft.ifft(self.Psi)

        # Final half-step in real space
        self.Psi *= self.R



In [ ]:


params = {'box_length': 500, 'v_offset': 0, 'grid_points': 1000, 'wfc_offset': 0, 'system': 'pib', 'mass': 1.0,
          'time_step': 1.0}
wf = Quantum(params)
ci = 0+1j
x0 = 200
sig = 15
k0 = 0.4*1

### T1 will be the prefactor that is 1/(sigma * sqrt(2 * pi))
T1 = 1/(sig * np.sqrt(2 * np.pi))
### T2 will be the Gaussian function, exp(-0.5 * ((x-x0)/sigma)^2)
T2 = np.exp(-0.5 * ((wf.x-x0)/sig)**2)
### T3 will be the complex exponential (aka the plane wave!)
T3 = np.exp(ci * k0 * wf.x)

wf.Psi = T1 * T2 * T3

### put in a square barrier between x=300 and x=400
wf.V[600:650] = -1.0

wf.build_operator()
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.close()


### parameters for plot
ax.set_xlim(( 0, 500))
ax.set_ylim((-0.003, 0.003))

line, = ax.plot([], [], lw=2)
lineV, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    lineV.set_data([], [])
    return (line, lineV,)

def animate(i):
  wf.split_op()
  #print(i,wf.Psi[2000])
  line.set_data(wf.x, np.real(np.conj(wf.Psi)*wf.Psi))
  lineV.set_data(wf.x, wf.V)
  return(line, lineV,)


anim = animation.FuncAnimation(fig, animate, init_func=init,
                             frames=400, interval=10, blit=True)

# Note: below is the part which makes it work on Colab
rc('animation', html='jshtml')
anim